In [1]:
from sklearn.cluster import KMeans
import numpy as np
import csv
import math
import matplotlib.pyplot
import pandas as pd
from matplotlib import pyplot as plt
from scipy import optimize as op

In [2]:
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10
C_Lambda = 0.03
IsSynthetic = False
M=10

In [3]:
def readPairs(filePath):    
    data = pd.read_csv(filePath) 
    #data.drop_duplicates(subset=['img_id_A'])
    return data
def readFeatures(filePath):    
    data = pd.read_csv(filePath) 
    return data

In [4]:
Human_Features_Data = readFeatures('HumanObserved-Features-Data.csv')
Diff_Pairs_Data = readPairs('diffn_pairs.csv')
Same_Pairs_Data=readPairs('same_pairs.csv')

# Concatenation and Subtraction of data for Human Observed Features

In [5]:
Data=pd.concat([Same_Pairs_Data,Diff_Pairs_Data.head(791)],axis=0)#concatenating same and different pairs for  human data set

Sample1= pd.merge(Data, Human_Features_Data, left_on='img_id_A' , right_on='img_id')
Sample2= pd.merge(Sample1,Human_Features_Data, left_on='img_id_B', right_on='img_id')
Datahumanfinal1=Sample2.sample(frac=1).reset_index(drop=True)
Datahumantarget=Datahumanfinal1.loc[:,'target']
Datahuman2=Datahumanfinal1.drop(['img_id_A','img_id_B','Unnamed: 0_x','Unnamed: 0_y','img_id_x','img_id_y','target'],axis =1)
Datahumanfinal=np.transpose(np.array(Datahuman2))
print(Datahumanfinal.shape)
print(Datahumantarget.shape)
#Subtracting same and different pairs for Human data set
Datahumansub=Datahumanfinal1.drop(['img_id_A','img_id_B','Unnamed: 0_x','Unnamed: 0_y','img_id_x','img_id_y','target'],axis =1)
for i in range (1,10):
    Datahumansub['f'+ str(i)]=Datahumansub['f'+str(i)+'_x']-Datahumansub['f'+str(i)+'_y']
    #datahumansubf=np.array(datahumansub)
for i in range (1,10):
    Datahumansub=Datahumansub.drop(['f'+str(i)+'_x','f'+str(i)+'_y'],axis =1)
Datahumansubf=np.transpose(np.array(Datahumansub))
print(Datahumansubf.shape)


(18, 1582)
(1582,)
(9, 1582)


# Concatenation and Difference of Data for GSC DataSet

In [6]:
GSC_Features_Data = readFeatures('GSC-Features.csv')
Diff_Pairs_Gsc = readPairs('diffn_pairsgsc.csv')
Same_Pairs_Gsc=readPairs('same_pairsgsc.csv')

In [7]:
Data_Gsc=pd.concat([Same_Pairs_Gsc.head(5000),Diff_Pairs_Gsc.head(5000)],axis=0)
#concatenating same and different pairs of GSC data set
Sample1_Gsc= pd.merge(Data_Gsc, GSC_Features_Data, left_on='img_id_A' , right_on='img_id')
Sample2_Gsc= pd.merge(Sample1_Gsc,GSC_Features_Data, left_on='img_id_B', right_on='img_id')
Datafinal1_Gsc=Sample2_Gsc.sample(frac=1).reset_index(drop=True)
Datatarget_Gsc=Datafinal1_Gsc.loc[:,'target']
#print(Datafinal1_Gsc)
DataGsc2=Datafinal1_Gsc.drop(['img_id_A','img_id_B','img_id_x','img_id_y','target'],axis =1)
DataGscfinal=np.transpose(np.array(DataGsc2))
print(DataGscfinal.shape)
print(Datatarget_Gsc.shape)
#subtracting same and different pairs in GSC data set
DataGscsub=Datafinal1_Gsc.drop(['img_id_A','img_id_B','img_id_x','img_id_y','target'],axis =1)
A,B=np.hsplit(DataGscsub,2)
C=np.subtract(B,A)
DataGsc2=pd.DataFrame(C)
DataGscsubf=np.transpose(np.array(DataGsc2))
print(DataGscsubf.shape)    


(1024, 10000)
(10000,)
(512, 10000)


In [8]:
def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    #returns an integer greater than or equal to the give number
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
#Traversing from o to 80 percent of  the data.
    d2 = rawData[:,0:T_len]
    print(d2.shape)
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2

def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
#Traversing from the next row of the Training data to the next 10 percent of the data
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    print(dataMatrix.shape)
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
#Traversing from the next row of the validation data to the next 10 percent of the data
    t =rawData[TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t
#Generating covariance matrix which is a diagonal matrix 
def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic):
    #creating a zero matrix with the dimensions as the data matrix
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)
    #training perdcent varies training,testing or validation
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))
    
    varVect     = []
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    #print(np.asarray(varVect).shape)
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:
    #dot product of vectors
        BigSigma = np.dot(3,BigSigma)
    else:
        BigSigma = np.dot(200,BigSigma)
    ##print ("BigSigma Generated..")
    return BigSigma

def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow)
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L

def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x


def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    #Transposing the data so that matrix multiplication is possible and we get a scalar values as output 
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    #BigSigma is 41*41 matrix
    BigSigInv = np.linalg.inv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
    #using gaussian radial basis function.
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI

def GetWeightsClosedForm(PHI, T, Lambda):
    #Identity array with one in the main daigonal
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    #print(Lambda)
    #getting weights using Moore penrose formula.
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    #print(PHI_SQR_LI)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    W           = np.dot(INTER, T)
    ##print ("Training Weights Generated..")
    return W



def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    ##print ("Test Out Generated..")
    return Y

def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    #len(VAL_TEST_OUT)
    #print(accuracy)
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))
  

# Partitioning Human Features Dataset

In [27]:
RawTarget = Datatarget_Gsc#this is passed as argument for target data in linear regression/logistic regression
RawData   = DataGscsubf#this is passed as argument for raw data in linear regression/logistic regression


In [28]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)

(512, 8000)


In [29]:
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
ValData    =GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))


(512, 999)


In [30]:
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))

(512, 999)


# Linear Regression Human Features Data Concatenation

In [36]:
ErmsArr = []
AccuracyArr = []

#finding the value of Mu by using kmeans clusters.
kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData))
Mu = kmeans.cluster_centers_

BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)
TRAINING_PHI = GetPhiMatrixGsc(RawData, Mu, BigSigma, TrainingPercent)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
TEST_PHI     = GetPhiMatrix(TestData, Mu, BigSigma, 100) 
VAL_PHI      = GetPhiMatrix(ValData, Mu, BigSigma, 100)

LinAlgError: Singular matrix

In [ ]:
print(Mu.shape)
print(BigSigma.shape)
print(TRAINING_PHI.shape)
print(W.shape)
print(VAL_PHI.shape)
print(TEST_PHI.shape)

In [15]:
W_Now        = np.dot(W,220)
La           = 2
learningRate = 0.03#this hyperparameter changes accuracy substantially. 
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []

for i in range(0,400):
    #print ('---------Iteration: ' + str(i) + '--------------')
    Delta_E_D     =-np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next)
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))

In [47]:
print ('----------Linear Regression Feature Concatenation--------------------')
print ('UBITname      = adhikary')
print ('Person Number = 50289864')
print ("M = 15 \nLambda  = 0.0001\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Linear Regression Feature Concatenation--------------------
UBITname      = adhikary
Person Number = 50289864
M = 15 
Lambda  = 0.0001
eta=0.01
E_rms Training   = 0.49252
E_rms Validation = 0.49866
E_rms Testing    = 0.49114


# Logistic Regression for Human Features Data Concatenation

In [32]:
W_Now        = np.random.rand(10,)
La           = 2
learningRate = 0.03
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []
for i in range(1266):
    #print ('---------Iteration: ' + str(i) + '--------------')
    Z = np.dot(np.transpose(W_Now),TRAINING_PHI[i])
    y_hat = 1/(1 + np.exp(-Z))
    Delta_E_D     =-np.dot((TrainingTarget[i] - y_hat),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

  #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next)
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))



In [49]:
print ('----------Logistic Regression Human Concatenation--------------------')
print ('UBITname      = adhikary')
print ('Person Number = 50289864')
print ("M = 15 \nLambda  = 0.0001\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Logistic Regression Human Concatenation--------------------
UBITname      = adhikary
Person Number = 50289864
M = 15 
Lambda  = 0.0001
eta=0.01
E_rms Training   = 0.49252
E_rms Validation = 0.49866
E_rms Testing    = 0.49114


# Partioning Data for Human Features Data

In [19]:
RawTarget = Datahumantarget
RawData   = Datahumansubf
#Training Data
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
#Validation Data
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
ValData    =GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))
#Test Data
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))

(9, 1266)
(9, 158)
(9, 157)


# Linear Regression for Human Features DataSet -Difference

In [20]:
ErmsArr = []
AccuracyArr = []

#finding the value of Mu by using kmeans clusters.
kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData))
Mu = kmeans.cluster_centers_

BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)
TRAINING_PHI = GetPhiMatrix(RawData, Mu, BigSigma, TrainingPercent)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
TEST_PHI     = GetPhiMatrix(TestData, Mu, BigSigma, 100) 
VAL_PHI      = GetPhiMatrix(ValData, Mu, BigSigma, 100)

In [21]:
print(Mu.shape)
print(BigSigma.shape)
print(TRAINING_PHI.shape)
print(W.shape)
print(VAL_PHI.shape)
print(TEST_PHI.shape)

(10, 9)
(9, 9)
(1266, 10)
(10,)
(158, 10)
(157, 10)


In [22]:
W_Now        = np.dot(W,220)
La           = 2
learningRate = 0.03
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []

for i in range(0,400):
    #print ('---------Iteration: ' + str(i) + '--------------')
    Delta_E_D     =-np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next)
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))

In [38]:
print ('----------Linear Regression Human Features Difference--------------------')
print ('UBITname      = adhikary')
print ('Person Number = 50289864')
print ("M = 15 \nLambda  = 0.0001\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Linear Regression Human Features Difference--------------------
UBITname      = adhikary
Person Number = 50289864
M = 15 
Lambda  = 0.0001
eta=0.01
E_rms Training   = 0.55236
E_rms Validation = 0.5609
E_rms Testing    = 0.55265


# Logistic Regression Human Features -Difference

In [24]:
W_Now        = np.random.rand(10,)
La           = 2
learningRate = 0.03
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []
for i in range(1266):
    #print ('---------Iteration: ' + str(i) + '--------------')
    Z = np.dot(np.transpose(W_Now),TRAINING_PHI[i])#in logistic regression, we use sigmoid function for activation. It classifies data as 0 and 1 in two classes.
    y_hat = 1/(1 + np.exp(-Z))#introducing sigmoid function as activation function here. Rest steps are similar as linear regression.
    Delta_E_D     =-np.dot((TrainingTarget[i] - y_hat),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next)
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))

    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))

    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))



In [39]:
print ('----------Logistic Regression Human Difference--------------------')
print ('UBITname      = adhikary')
print ('Person Number = 50289864')
print ("M = 15 \nLambda  = 0.0001\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Logistic Regression Human Difference--------------------
UBITname      = adhikary
Person Number = 50289864
M = 15 
Lambda  = 0.0001
eta=0.01
E_rms Training   = 0.55236
E_rms Validation = 0.5609
E_rms Testing    = 0.55265


# Partioning Data GscFeatures-Concatenation

In [40]:
RawTarget = Datatarget_Gsc
RawData   = DataGscfinal
#Training Data
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
#Validation Data
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
ValData    =GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))
#Test Data
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))

(1024, 8000)
(1024, 999)
(1024, 999)


# Linear Regression Gsc Features-Concatenation

In [41]:
def GetPhiMatrixGsc(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    #Transposing the data so that matrix multiplication is possible and we get a scalar values as output 
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    #BigSigma is 41*41 matrix
    BigSigInv = np.linalg.pinv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
    #using gaussian radial basis function.
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI

In [16]:
ErmsArr = []
AccuracyArr = []

#finding the value of Mu by using kmeans clusters.
kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData))
Mu = kmeans.cluster_centers_

BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)
TRAINING_PHI = GetPhiMatrixGsc(RawData, Mu, BigSigma, TrainingPercent)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
TEST_PHI     = GetPhiMatrixGsc(TestData, Mu, BigSigma, 100) 
VAL_PHI      = GetPhiMatrixGsc(ValData, Mu, BigSigma, 100)

#Gradient Descent Solution
W_Now        = np.dot(W,220)
La           = 2
learningRate = 0.03
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []

for i in range(0,400):
    #print ('---------Iteration: ' + str(i) + '--------------')
    Delta_E_D     =-np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next)
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))

In [42]:
print ('----------Linear Regression GSC  Features Concatenation--------------------')
print ('UBITname      = adhikary')
print ('Person Number = 50289864')
print ("M = 15 \nLambda  = 0.0001\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Linear Regression GSC  Features Concatination--------------------
UBITname      = adhikary
Person Number = 50289864
M = 15 
Lambda  = 0.0001
eta=0.01
E_rms Training   = 0.55236
E_rms Validation = 0.5609
E_rms Testing    = 0.55265


# Logistic Regression Gsc Features -Concatenation

In [18]:
W_Now        = np.random.rand(10,)
La           = 2
learningRate = 0.03
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []
for i in range(400):
    #print ('---------Iteration: ' + str(i) + '--------------')
    Z = np.dot(np.transpose(W_Now),TRAINING_PHI[i])
    y_hat = 1/(1 + np.exp(-Z))
    Delta_E_D     =-np.dot((TrainingTarget[i] - y_hat),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next)
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))

    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))

    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))



In [43]:
print ('----------Logistic GSC  Features Concatenation--------------------')
print ('UBITname      = adhikary')
print ('Person Number = 50289864')
print ("M = 15 \nLambda  = 0.0001\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Logistic GSC  Features Concatination--------------------
UBITname      = adhikary
Person Number = 50289864
M = 15 
Lambda  = 0.0001
eta=0.01
E_rms Training   = 0.55236
E_rms Validation = 0.5609
E_rms Testing    = 0.55265


# Partioning Data Gsc Features -Difference

In [20]:
RawTarget = Datatarget_Gsc
RawData   = DataGscsubf
#Training Data
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)
#Validation Data
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
ValData    =GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))
#Test Data
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))

(512, 8000)
(512, 999)
(512, 999)


# Linear Regression Gsc Features-Difference

In [44]:
ErmsArr = []
AccuracyArr = []

#finding the value of Mu by using kmeans clusters.
kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData))
Mu = kmeans.cluster_centers_

BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)
TRAINING_PHI = GetPhiMatrixGsc(RawData, Mu, BigSigma, TrainingPercent)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
TEST_PHI     = GetPhiMatrixGsc(TestData, Mu, BigSigma, 100) 
VAL_PHI      = GetPhiMatrixGsc(ValData, Mu, BigSigma, 100)

#Gradient Descent Solution
W_Now        = np.dot(W,220)
La           = 2
learningRate = 0.03
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []

for i in range(0,400):
    #print ('---------Iteration: ' + str(i) + '--------------')
    Delta_E_D     =-np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next)
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))

In [45]:
print ('----------Linear Regression GSC  Features Difference--------------------')
print ('UBITname      = adhikary')
print ('Person Number = 50289864')
print ("M = 15 \nLambda  = 0.0001\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Linear Regression GSC  Features Difference--------------------
UBITname      = adhikary
Person Number = 50289864
M = 15 
Lambda  = 0.0001
eta=0.01
E_rms Training   = 0.49252
E_rms Validation = 0.49866
E_rms Testing    = 0.49114


# Logistic Regression Gsc Features-Subtraction

In [ ]:
W_Now        = np.random.rand(10,)
La           = 2
learningRate = 0.03
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []
for i in range(400):
    #print ('---------Iteration: ' + str(i) + '--------------')
    Z = np.dot(np.transpose(W_Now),TRAINING_PHI[i])
    y_hat = 1/(1 + np.exp(-Z))
    Delta_E_D     =-np.dot((TrainingTarget[i] - y_hat),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next

    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next)
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))

    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))

    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))

#lossfunction=lossfunction(y_hat,TrainingTarget)
#def lossfunction(y_hat, TrainingTarget):
#    return(-TrainingTarget[i]*np.log(h)=(1-y)*np.log(1-h)).mean()
#lossfunction=lossfunction(y_hat,TrainingTarget)
#print(lossfunction(y_hat, TrainingTarget))

In [46]:
print ('----------Logistic GSC  Features Difference--------------------')
print ('UBITname      = adhikary')
print ('Person Number = 50289864')
print ("M = 15 \nLambda  = 0.0001\neta=0.01")
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))

----------Logistic GSC  Features Difference--------------------
UBITname      = adhikary
Person Number = 50289864
M = 15 
Lambda  = 0.0001
eta=0.01
E_rms Training   = 0.49252
E_rms Validation = 0.49866
E_rms Testing    = 0.49114
